In [19]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import random
from datetime import datetime
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
# Import the API key.
from config import weather_api_key

In [20]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90., high=90., size=200)
lngs = np.random.uniform(low=-180., high=180., size=200)
lat_lngs = zip(lats, lngs)
lat_lngs

In [21]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [22]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [23]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

146

In [24]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=14d496981f3ab4ab4fbc15adae63ab96


In [32]:
city_temp = cities[0:20]

In [44]:
# Create an empty list to hold the weather data.
city_data = [] 
cdata=[]
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(city_temp):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        print(city_lat)
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]("{:.0f}".format)
        city_clouds = city_weather["clouds"]["all"]("{:.0f}".format)
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description,
                         })
        cdata.append(city_lat)

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tasiilaq
65.6145
City not found. Skipping...
Processing Record 2 of Set 1 | baykit
61.67
City not found. Skipping...
Processing Record 3 of Set 1 | purworejo
-6.7872
City not found. Skipping...
Processing Record 4 of Set 1 | grindavik
63.8424
City not found. Skipping...
Processing Record 5 of Set 1 | tuktoyaktuk
69.4541
City not found. Skipping...
Processing Record 6 of Set 1 | samana
19.2056
City not found. Skipping...
Processing Record 7 of Set 1 | marau
-28.4492
City not found. Skipping...
Processing Record 8 of Set 1 | bhatapara
21.7333
City not found. Skipping...
Processing Record 9 of Set 1 | port alfred
-33.5906
City not found. Skipping...
Processing Record 10 of Set 1 | hermanus
-34.4187
City not found. Skipping...
Processing Record 11 of Set 1 | qaanaaq
77.484
City not found. Skipping...
Processing Record 12 of Set 1 | tura
25.5198
City not found. Skipping...
Processing Record 13 of Set 

In [40]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

""


In [38]:
city_data_df.to_csv('WeatherPy_Database.csv')

In [41]:
city_data

[]

In [45]:
cdata

[]